In [3]:
import numpy as np
import pandas as pd
import csv
import math
import pickle
import import_ipynb
import pattern_count
import time

In [4]:
def P1DominatedByP2(P1, P2):
    length = len(P1)
    for i in range(length):
        if P1[i] == -1:
            if P2[i] != -1:
                return False
        if P1[i] != -1:
            if P2[i] != P1[i] and P2[i] != -1:
                return False
    return True

# whether a pattern P is dominated by MUP M
def PDominatedByM(P, M):
    for m in M:
        if P1DominatedByP2(P, m):
            return True
    return False

# whether a pattern P dominates MUP M
def PDominatesM(P, M):
    for m in M:
        if P1DominatedByP2(m, P):
            return True
    return False


# coverage of P among dataset D
def cov(P, D):
    cnt = 0
    for d in D:
        if P1DominatedByP2(d, P):
            cnt += 1
    return cnt


def GenerateParents(P):
    parents = []
    length = len(P)
    for i in range(length):
        if P[i] != -1:
            q = P.copy()
            q[i] = -1
            parents.append(q)
    return parents

def GenerateByRule1(P, mcdes, attributes):
    children = []
    length = len(P)
    i = 0
    for i in range(length-1, -1, -1):
        if P[i] != -1:
            break
    if P[i] == -1:
        i -= 1
    for j in range(i+1, length, 1):
        for a in range(int(mcdes[attributes[j]]['min']), int(mcdes[attributes[j]]['max'])+1):
            s = P.copy()
            s[j] = a
            children.append(s)
    return children


def num2string(pattern):
    st = ''
    for i in pattern:
        if i != -1:
            st += str(i)
        st += '|'
    st = st[:-1]
    return st



def Deepdiver(file, tao):
    mc, mcdes, attributes = Prepatation(file)
    column_list = np.array(mc.columns).tolist()
    pc = pattern_count.PatternCounter(file, column_list, encoded=False)
    pc.parse_data()
    mcarray = np.array(mc)
    mclist = mcarray.tolist()
    
    root = [-1] * (len(attributes))
    S = [root]  # initial stack
    M = []  # maximal uncovered patterns
    while len(S) > 0:
        P = S.pop()
        #print(P, len(M))
        if PDominatedByM(P, M):
            continue
        elif PDominatesM(P, M):
            uncoveredFlag = True
        else:
            cnt = cov(P, mclist)
            #uncoveredFlag = (cnt < tao and cnt > 0) # add > 0 requirement here
            uncoveredFlag = cnt < tao 
        if uncoveredFlag:
            S1 = [P] # stack
            while len(S1) > 0:
                P1 = S1.pop()
                ParentNodes = GenerateParents(P1)
                for parent in ParentNodes:
                    cntparent = cov(parent, mclist)
                    if cntparent < tao:
                        S1.append(parent)
                        break
                M.append(P)
                if len(M) % 1000 == 0:
                    print(len(M))
            # end while
        else:
            children = GenerateByRule1(P, mcdes, attributes)
            S = S + children
    return M


def Prepatation(filename):
    mc = pd.read_csv(filename)
    mcdes = mc.describe()
    attributes = mcdes.columns.values
    return mc, mcdes, attributes


In [5]:

mc, mcdes, attributes = Prepatation('miss_class3.csv')
print(len(attributes))
mcarray = np.array(mc)
mclist = mcarray.tolist()


13


In [25]:
time1 = time.time()
M = Deepdiver('miss_class3.csv', 50*0.6)
time2 = time.time()
print("--- %s seconds ---" % (time2 - time1))
print(M)

1000
2000
3000
4000
5000
6000
7000


NameError: name 'start_time' is not defined